In [1]:
#!/usr/bin/env python3
"""
Live Overlay Strategy Signal Generator
Fetches live Nifty 50 and Midcap data and provides trading signals
Uses multiple data sources for real-time data
"""

import yfinance as yf
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import requests
import warnings
import time
warnings.filterwarnings('ignore')

# ---------------------------
# Configuration
# ---------------------------
NIFTY_SYMBOL = "^NSEI"      # Nifty 50
MIDCAP_SYMBOL = "NIFTYMIDCAP150.NS"  # Nifty Midcap 150
LOOKBACK_DAYS = 400         # Increased for better historical data
SLOPE_PERIOD = 63           # ~3 months for slope calculation

# Portfolio parameters
LOT_SIZE = 50
FUT_BASIS = 0.002
TAX_RATE = 0.30

# NSE API URLs for live data
NSE_HEADERS = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36',
    'Accept': 'application/json',
    'Accept-Language': 'en-US,en;q=0.9',
    'Accept-Encoding': 'gzip, deflate, br',
    'Connection': 'keep-alive',
    'Upgrade-Insecure-Requests': '1',
}

def get_live_nse_data():
    """Fetch live NSE index data directly from NSE website"""
    try:
        # Session for maintaining cookies
        session = requests.Session()
        session.headers.update(NSE_HEADERS)
        
        # Get main page first to establish session
        session.get('https://www.nseindia.com', timeout=10)
        
        # Fetch Nifty 50 data
        nifty_url = "https://www.nseindia.com/api/equity-stockIndices?index=NIFTY%2050"
        response = session.get(nifty_url, timeout=10)
        
        if response.status_code == 200:
            data = response.json()
            if data.get('data') and len(data['data']) > 0:
                nifty_data = data['data'][0]
                nifty_price = float(nifty_data['last'])
                nifty_time = data.get('timestamp', datetime.now().strftime('%d-%b-%Y %H:%M:%S'))
                print(f"🟢 Live Nifty 50: ₹{nifty_price:,.2f} at {nifty_time}")
                return nifty_price, nifty_time
        
    except Exception as e:
        print(f"⚠️  NSE API failed: {e}")
    
    return None, None

def get_live_alternative_data(symbol):
    """Alternative method using yfinance with extended period"""
    try:
        ticker = yf.Ticker(symbol)
        
        # Try to get today's data first
        today_data = ticker.history(period="1d", interval="1m")
        if not today_data.empty:
            current_price = today_data['Close'].iloc[-1]
            current_time = today_data.index[-1].strftime('%Y-%m-%d %H:%M:%S')
            return current_price, current_time
            
        # Fallback to recent data
        recent_data = ticker.history(period="5d")
        if not recent_data.empty:
            current_price = recent_data['Close'].iloc[-1]
            current_time = recent_data.index[-1].strftime('%Y-%m-%d %H:%M:%S')
            return current_price, current_time
            
    except Exception as e:
        print(f"⚠️  Alternative data fetch failed: {e}")
    
    return None, None

# ---------------------------
# Strategy Decision Function (Core Strategy - Unchanged)
# ---------------------------
def overlay_decision(spot, dma200, slope):
    """
    Determine overlay strategy based on current conditions
    Returns: (beta, regime, action_description)
    """
    if np.isnan(dma200) or dma200 is None:
        return 0.0, "No DMA", "⚪ WAIT - Not enough data for 200 DMA"
    
    gap = (spot - dma200) / dma200
    
    if gap >= 0.04:
        return 1.5, "Strong up-trend", "🟢 GO LONG - Add 50% futures long position"
    elif gap <= -0.08 and slope < 0:
        return -0.50, "Deep bear", "🔴 GO SHORT - Open 50% futures short position"
    elif gap <= -0.04:
        return -0.25, "Correction", "🟠 GO SHORT - Open 25% futures short position"
    else:
        return 0.0, "Neutral", "⚪ DO NOTHING - Stay in cash/equity only"

# ---------------------------
# Enhanced Data Fetching Function
# ---------------------------
def fetch_live_data(symbol, days=LOOKBACK_DAYS):
    """Fetch historical + live data using multiple sources"""
    try:
        end_date = datetime.now()
        start_date = end_date - timedelta(days=days)
        
        # Get historical data for calculations
        ticker = yf.Ticker(symbol)
        hist_data = ticker.history(start=start_date, end=end_date)
        
        if hist_data.empty:
            raise Exception(f"No historical data found for {symbol}")
        
        # Try to get today's live price
        live_price = None
        live_time = None
        
        if symbol == "^NSEI":
            live_price, live_time = get_live_nse_data()
        
        # If NSE API failed or for other symbols, try alternative
        if live_price is None:
            live_price, live_time = get_live_alternative_data(symbol)
        
        # Update with live price if available
        if live_price is not None and live_time is not None and live_price > 0:
            today = datetime.now().date()
            last_date = hist_data.index[-1].date()
            
            if today > last_date:
                try:
                    new_row = hist_data.iloc[-1].copy()
                    new_row['Close'] = live_price
                    new_row['High'] = max(float(new_row['High']), live_price)
                    new_row['Low'] = min(float(new_row['Low']), live_price)
                    
                    today_index = pd.Timestamp(today)
                    hist_data.loc[today_index] = new_row
                except Exception:
                    pass
            else:
                try:
                    hist_data.iloc[-1, hist_data.columns.get_loc('Close')] = live_price
                except Exception:
                    pass
        
        return hist_data
        
    except Exception as e:
        print(f"❌ Error fetching data for {symbol}: {e}")
        return None

# ---------------------------
# Technical Analysis
# ---------------------------
def calculate_indicators(data):
    """Calculate 200 DMA and slope"""
    if data is None or len(data) == 0:
        return None
        
    data = data.copy()
    data['200DMA'] = data['Close'].rolling(window=200, min_periods=200).mean()
    
    # Calculate slope more safely
    if len(data) >= SLOPE_PERIOD:
        data['200DMA_slope'] = data['200DMA'].diff(SLOPE_PERIOD)
    else:
        data['200DMA_slope'] = 0.0
        
    return data

# ---------------------------
# Portfolio Position Calculator
# ---------------------------
def calculate_position_size(beta, portfolio_allocation, current_price):
    """Calculate exact futures position for given portfolio allocation"""
    if beta == 0 or portfolio_allocation <= 0 or current_price <= 0:
        return 0, 0
        
    try:
        fut_price = current_price * (1 + FUT_BASIS)
        contract_notional = fut_price * LOT_SIZE
        
        if contract_notional <= 0:
            return 0, 0
            
        lots_needed = int(round(abs(beta) * portfolio_allocation / contract_notional))
        total_notional = lots_needed * contract_notional
        
        return lots_needed, total_notional
    except Exception as e:
        print(f"⚠️  Error calculating position size: {e}")
        return 0, 0

# ---------------------------
# Timing Validation Function
# ---------------------------
def check_optimal_timing():
    """Check if running at optimal time (last 30 min of month-end)"""
    now = datetime.now()
    
    # Get last trading day of current month
    next_month = now.replace(day=28) + timedelta(days=4)
    last_day_of_month = next_month - timedelta(days=next_month.day)
    
    # Check if it's within last 30 minutes of trading (assuming 3:30 PM close)
    market_close = last_day_of_month.replace(hour=15, minute=30, second=0, microsecond=0)
    optimal_start = market_close - timedelta(minutes=30)
    
    is_last_day = now.date() == last_day_of_month.date()
    is_optimal_time = is_last_day and optimal_start <= now <= market_close
    
    if not is_optimal_time:
        if now.weekday() < 5:  # Weekday
            print("⚠️  TIMING WARNING:")
            print("   This strategy is designed to run during the last 30 minutes")
            print("   of the final trading day of each month for optimal results.")
            if not is_last_day:
                print(f"   Next optimal run: {last_day_of_month.strftime('%d %b %Y')} between 3:00-3:30 PM")
        print()

def get_short_action(beta):
    """Get shortened action description"""
    try:
        if beta > 0:
            return f"🟢 LONG {abs(beta):.1f}x"
        elif beta < 0:
            return f"🔴 SHORT {abs(beta):.1f}x"
        else:
            return "⚪ NEUTRAL"
    except:
        return "⚪ ERROR"

# ---------------------------
# Main Analysis Function (Streamlined)
# ---------------------------
def analyze_index(symbol, index_name, show_details=False):
    """Analyze single index and return signal"""
    try:
        # Fetch data quietly
        data = fetch_live_data(symbol)
        if data is None or len(data) == 0:
            return None
            
        # Calculate indicators
        data = calculate_indicators(data)
        if data is None:
            return None
        
        # Get latest values
        latest = data.iloc[-1]
        current_price = float(latest['Close'])
        dma200 = latest['200DMA']
        slope = latest['200DMA_slope'] if pd.notna(latest['200DMA_slope']) else 0
        current_date = latest.name.strftime('%Y-%m-%d')
        
        # Check if we have enough data
        if pd.isna(dma200) or dma200 <= 0:
            return None
        
        dma200 = float(dma200)
        slope = float(slope)
        
        # Calculate gap
        gap_pct = (current_price - dma200) / dma200 * 100
        
        # Get strategy decision
        beta, regime, action = overlay_decision(current_price, dma200, slope)
        
        return {
            'symbol': symbol,
            'index_name': index_name,
            'price': current_price,
            'dma200': dma200,
            'gap_pct': gap_pct,
            'slope': slope,
            'regime': regime,
            'beta': beta,
            'action': action,
            'timestamp': current_date
        }
        
    except Exception as e:
        if show_details:
            print(f"❌ Error analyzing {symbol}: {e}")
        return None

# ---------------------------
# Input Validation Function
# ---------------------------
def get_portfolio_value():
    """Get and validate portfolio value from user"""
    while True:
        try:
            portfolio_input = input("💰 Portfolio value (₹) [default: 10,00,000]: ").strip()
            if not portfolio_input:
                return 10_00_000
            
            # Clean input
            cleaned_input = portfolio_input.replace(",", "").replace("₹", "").replace(" ", "")
            portfolio_value = int(float(cleaned_input))
            
            if portfolio_value <= 0:
                print("❌ Portfolio value must be positive. Please try again.")
                continue
                
            if portfolio_value < 100000:  # Less than 1 lakh
                confirm = input(f"⚠️  Portfolio value is ₹{portfolio_value:,}. Confirm? (y/n): ").lower()
                if confirm != 'y':
                    continue
            
            return portfolio_value
            
        except ValueError:
            print("❌ Invalid input. Please enter a valid number.")
        except KeyboardInterrupt:
            print("\n👋 Exiting...")
            exit()
        except Exception as e:
            print(f"❌ Error: {e}. Please try again.")

# ---------------------------
# Main Execution
# ---------------------------
def main():
    """Main execution function"""
    try:
        print("🚀 OVERLAY STRATEGY ANALYZER")
        print(f"📅 {datetime.now().strftime('%d %b %Y, %H:%M')}")
        
        # Check optimal timing
        check_optimal_timing()
        
        # Get portfolio value
        portfolio_value = get_portfolio_value()
        
        # Calculate 50-50 allocation
        nifty_allocation = portfolio_value * 0.5
        midcap_allocation = portfolio_value * 0.5
        
        print("📡 Fetching data...\n")
        
        # Analyze indices
        nifty_result = analyze_index(NIFTY_SYMBOL, "Nifty 50")
        midcap_result = analyze_index(MIDCAP_SYMBOL, "Midcap 150")
        
        # Current Status Summary
        print("📊 CURRENT ANALYSIS (50% Nifty | 50% Midcap)")
        print("═" * 55)
        
        results = [
            (nifty_result, "NIFTY 50", nifty_allocation),
            (midcap_result, "MIDCAP 150", midcap_allocation)
        ]
        
        for result, name, allocation in results:
            if result:
                print(f"\n{name} (₹{allocation:,.0f} allocation):")
                print(f"  Price: ₹{result['price']:,.0f}")
                print(f"  200DMA Gap: {result['gap_pct']:+.1f}%")
                print(f"  Signal: {get_short_action(result['beta'])} ({result['regime']})")
            else:
                print(f"\n{name}: ❌ Data unavailable")
        
        # Position Sizing with 50-50 split
        print(f"\n🎯 FUTURES OVERLAY POSITIONS")
        print("═" * 55)
        print(f"Total Portfolio: ₹{portfolio_value:,} (50% Nifty | 50% Midcap)")
        print()
        
        total_margin = 0
        has_positions = False
        active_overlays = []
        
        position_details = [
            (nifty_result, "Nifty 50", nifty_allocation),
            (midcap_result, "Midcap 150", midcap_allocation)
        ]
        
        for result, index_name, allocation in position_details:
            if result and result['beta'] != 0:
                beta = result['beta']
                price = result['price']
                lots, notional = calculate_position_size(beta, allocation, price)
                
                if lots > 0:
                    direction = "LONG" if beta > 0 else "SHORT"
                    margin_est = notional * 0.10
                    total_margin += margin_est
                    has_positions = True
                    
                    overlay_strength = abs(beta) * 100
                    active_overlays.append(f"{index_name}: {direction} {overlay_strength:.0f}%")
                    
                    print(f"{index_name}:")
                    print(f"  {direction} {lots} lots @ {overlay_strength:.0f}% overlay")
                    print(f"  Margin: ₹{margin_est:,.0f}")
            else:
                print(f"{index_name}: No overlay (Stay in equity)")
        
        print()
        if has_positions:
            risk_pct = (total_margin / portfolio_value) * 100
            print(f"Total Margin Required: ₹{total_margin:,.0f} ({risk_pct:.1f}% of portfolio)")
            
            if risk_pct > 15:
                print("⚠️  HIGH RISK: Consider reducing overlay percentages")
            
            print(f"\nActive Overlays: {' | '.join(active_overlays)}")
        else:
            print("✅ No futures overlay needed - Stay 100% in underlying equities")
        
        # Caution Notice
        print(f"\n⚠️  IMPORTANT DISCLAIMERS")
        print("═" * 55)
        print("• This is for EDUCATIONAL purposes only")
        print("• 50-50 allocation assumes balanced Nifty-Midcap equity portfolio")
        print("• Futures overlay is ADDITIONAL to your equity positions")
        print("• Past performance does not guarantee future results")
        print("• Futures trading involves substantial risk of loss")
        print("• Consult qualified financial advisor before implementing")
        print("• Strategy requires professional risk management")
        print("• Market conditions can change rapidly")
        
    except KeyboardInterrupt:
        print("\n👋 Analysis stopped by user")
    except Exception as e:
        print(f"❌ Error: {e}")

# ---------------------------
# Alternative: Continuous Monitoring (Optional)
# ---------------------------
def monitor_continuous():
    """Optional function for continuous monitoring"""
    print("\n🔄 Starting continuous monitoring...")
    print("Press Ctrl+C to stop")
    
    try:
        while True:
            print(f"\n{'='*40}")
            print(f"📡 Refresh: {datetime.now().strftime('%H:%M:%S')}")
            
            result = analyze_index(NIFTY_SYMBOL, "Nifty 50")
            if result and result['beta'] != 0:
                print(f"🚨 ACTIVE SIGNAL: {get_short_action(result['beta'])}")
            else:
                print("✅ No action required")
                
            print("⏰ Next check in 5 minutes...")
            time.sleep(300)  # Wait 5 minutes
            
    except KeyboardInterrupt:
        print("\n🛑 Monitoring stopped")

if __name__ == "__main__":
    main()

# Uncomment below line to enable continuous monitoring
# monitor_continuous()

🚀 OVERLAY STRATEGY ANALYZER
📅 28 Aug 2025, 11:23
⚠️  TIMING WARNING:
   This strategy is designed to run during the last 30 minutes
   of the final trading day of each month for optimal results.
   Next optimal run: 31 Aug 2025 between 3:00-3:30 PM

💰 Portfolio value (₹) [default: 10,00,000]: 544556779
📡 Fetching data...

📊 CURRENT ANALYSIS (50% Nifty | 50% Midcap)
═══════════════════════════════════════════════════════

NIFTY 50 (₹272,278,390 allocation):
  Price: ₹24,533
  200DMA Gap: +1.9%
  Signal: ⚪ NEUTRAL (Neutral)

MIDCAP 150 (₹272,278,390 allocation):
  Price: ₹20,860
  200DMA Gap: +2.0%
  Signal: ⚪ NEUTRAL (Neutral)

🎯 FUTURES OVERLAY POSITIONS
═══════════════════════════════════════════════════════
Total Portfolio: ₹544,556,779 (50% Nifty | 50% Midcap)

Nifty 50: No overlay (Stay in equity)
Midcap 150: No overlay (Stay in equity)

✅ No futures overlay needed - Stay 100% in underlying equities

⚠️  IMPORTANT DISCLAIMERS
═══════════════════════════════════════════════════════
